In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

import add_features

from statsmodels.tsa.stattools import adfuller

%matplotlib inline

In [ ]:
c_dir = "C://Users/shlomi/Documents/Work/vindish/data/"
e_dir = "E:\\Work/Vindish/created_samples/"

In [ ]:
df = pd.read_csv(c_dir+"VIX Historical data 10 Min level since 2017 UX1_UX5.csv")

In [ ]:
df.head()

In [ ]:
df[df.UX1.isnull()].head()

In [ ]:
df.iloc[:, 2:7].plot()

In [ ]:
df.iloc[0:50000:5000, 2:7].T.plot()

In [ ]:
df = add_features.drop_rows_with_null_dates(df)

In [ ]:
df.Date = pd.to_datetime(df.Date, dayfirst=True)

In [ ]:
df = pd.concat([df, add_features.from_date_get_dow_dom_doy(df.Date)], axis=1)

In [ ]:
df.head()

In [ ]:
diff_cols = [i+"_diff" for i in df.iloc[:, 2:8].columns]
df[diff_cols] = df.iloc[:, 2:8].diff()
df = df.iloc[1:, :]

In [ ]:
df["day_of_month"] = df.Date.dt.day
df["day_of_week"] = df.Date.dt.dayofweek
df["day_of_year"] = df.Date.dt.dayofyear

In [ ]:
df_dow = pd.get_dummies(df.day_of_week)
df_dow_columns = ["dow_"+str(i) for i in df.day_of_week.unique().tolist()]
df_dow.columns = df_dow_columns
# df_dow
df_dom = pd.get_dummies(df.day_of_month)
df_dom_columns = ["dom_"+str(i) for i in df.day_of_month.unique().tolist()]
df_dom.columns = df_dom_columns

In [ ]:
df = pd.concat([df, df_dow, df_dom], axis=1)

In [ ]:
df.memory_usage().sum()//1000000

In [ ]:
rol = df.iloc[:, 2:7].rolling(window=1000)

In [ ]:
x = df.iloc[:11]

In [ ]:
rol.mean().plot()

In [ ]:
ux_vals = x[["UX1", "UX2", "UX3", "UX4", "UX5"]].iloc[:10].values
ux_vals

In [ ]:
dow_dummy = pd.get_dummies(x.day_of_week)#, columns=days_list)

In [ ]:
df.columns

In [ ]:
def create_xy_from_df(x, verbose=False):
    time_to_expiration = np.expand_dims(x.iloc[:10].loc[:, "Time_To_Expiration"].values, 1)
    ux_vals = x.iloc[:10].loc[:, ['UX1', 'UX2', 'UX3', 'UX4','UX5']].values
    snp_vals = np.expand_dims(x.iloc[:10].loc[:, "SP500"].values, 1)
    day_of_year = np.expand_dims(x.iloc[:10].loc[:, "day_of_year"].values, 1)

    day_of_week_dummies = x.iloc[:10].loc[:, df_dow_columns].values

    day_of_month_dummies = x.iloc[:10].loc[:, df_dom_columns].values

    diffs = x.iloc[:10].loc[:, diff_cols[:-1]].values
    diff_snp = np.expand_dims(x.iloc[:10].loc[:, diff_cols[-1]].values, 1)

    if verbose:
        print("ux_vals.shape: ", ux_vals.shape)
        print("diffs.shape: ", diffs.shape)
        print("snp_vals.shape: ", snp_vals.shape)
        print("diff_snp.shape: ", diff_snp.shape)
        print("time_to_expiration.shape: ", time_to_expiration.shape)
        print("day_of_year.shape: ", day_of_year.shape)
        print("day_of_week_dummies.shape: ", day_of_week_dummies.shape)
        print("day_of_month_dummies.shape: ", day_of_month_dummies.shape)

    X = np.concatenate([ux_vals, diffs, snp_vals, diff_snp, time_to_expiration, day_of_year, day_of_week_dummies, day_of_month_dummies], axis=1)
    y = x[["UX1", "UX2", "UX3", "UX4", "UX5"]].iloc[10].values
    
    return X,y

In [ ]:
i=11
X, y = create_xy_from_df(df.iloc[i-11:i, :], verbose=True)

In [ ]:
x = df.iloc[i-11:i, :]
dow_list = df.Date.dt.dayofweek.unique().tolist()
print(dow_list)
x.iloc[:10]#.loc[:, dow_list].values

In [ ]:
X.shape

In [ ]:
df.Date.dt.dayofweek.unique().tolist()

In [ ]:
%%time
if os.path.exists(e_dir + "X.npy") and os.path.exists(e_dir + "y.npy"):
    X = np.load(e_dir + "X.npy")
    y = np.load(e_dir + "y.npy")
else:    
    Xs = []; ys=[]
    for i in tqdm(np.arange(11, df.shape[0]-1)):
    #     print(df.iloc[(i-11):i, :].shape)
        X, y= create_xy_from_df(df.iloc[(i-11):i, :])
        Xs.append(X)
        ys.append(y)
    X = np.stack(Xs)
    y = np.stack(ys)
    np.save(e_dir + "X.npy", X)
    np.save(e_dir + "y.npy", y)

In [ ]:
X.shape, y.shape

In [ ]:
pd.DataFrame(X[0])

In [ ]:
import keras
import torch

In [ ]:
x = X[0]

In [ ]:
def to_expanded_tensor(x):
    x = np.expand_dims(x, 0)
    x = np.expand_dims(x, 0)
    x = torch.Tensor(x)
    return x

In [ ]:
ux_vals = x[:,:5]
ux_vals = to_expanded_tensor(ux_vals)

ux_diffs = x[:, 5:10]
ux_diffs = to_expanded_tensor(ux_diffs)

snp_data = x[:, 10:12]
snp_data = to_expanded_tensor(snp_data)

In [ ]:
x.shape

In [ ]:
class MiniConv2d(torch.nn.Module):
    def __init__(self):
        super(MiniConv2d, self).__init__()
        conv1 = torch.nn.Conv2d(1, 5, 3, padding=1)
        conv2 = torch.nn.Conv2d(5, 5, 3, padding=1)
        conv3 = torch.nn.Conv2d(5, 5, 3, padding=1)
        conv4 = torch.nn.Conv2d(5, 5, 3, padding=1)
        conv5 = torch.nn.Conv2d(5, 5, 3, padding=1)
        conv6 = torch.nn.Conv2d(5, 5, 3, padding=1)
        conv7 = torch.nn.Conv2d(5, 5, 3, padding=1)
    def forward(self, x):
        x = conv1(x)
        x = torch.nn.functional.relu(x)
        x = conv2(x)
        x = torch.nn.functional.relu(x)
        x = conv3(x)
        x = torch.nn.functional.relu(x)
        x = conv4(x)
        x = torch.nn.functional.relu(x)
        x = conv5(x)
        x = torch.nn.functional.relu(x)
        x = conv6(x)
        x = torch.nn.functional.relu(x)
        x = conv7(x)
        x = torch.nn.functional.relu(x)
        
        return x

In [ ]:
class MiniConv1d(torch.nn.Module):
    def __init__(self):
        super(MiniConv1d, self).__init__()
        conv1 = torch.nn.Conv1d(1, 5, 3, padding=1)
        conv2 = torch.nn.Conv1d(5, 5, 3, padding=1)
        conv3 = torch.nn.Conv1d(5, 5, 3, padding=1)
        conv4 = torch.nn.Conv1d(5, 5, 3, padding=1)
        conv5 = torch.nn.Conv1d(5, 5, 3, padding=1)
        conv6 = torch.nn.Conv1d(5, 5, 3, padding=1)
        conv7 = torch.nn.Conv1d(5, 5, 3, padding=1)
    def forward(self, x):
        x = conv1(x)
        x = torch.nn.functional.relu(x)
        x = conv2(x)
        x = torch.nn.functional.relu(x)
        x = conv3(x)
        x = torch.nn.functional.relu(x)
        x = conv4(x)
        x = torch.nn.functional.relu(x)
        x = conv5(x)
        x = torch.nn.functional.relu(x)
        x = conv6(x)
        x = torch.nn.functional.relu(x)
        x = conv7(x)
        x = torch.nn.functional.relu(x)
        
        return x

In [ ]:
mini_conv_ux = MiniConv2d()
mini_conv_ux_diffs = MiniConv2d()
mini_conv_snp = MiniConv1d()

In [ ]:
mini_conv_ux(ux_vals).shape
mini_conv_ux_diffs(ux_diffs).shape
mini_conv_snp(snp_data).shape